
<a href="https://colab.research.google.com/github/kokchun/Machine-learning-AI22/blob/main/Exercises/E04_logistic_regression.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; to see hints and answers.

---
# Logistic regression exercises 

---
These are introductory exercises in Machine learning with focus in **logistic regression**

<p class = "alert alert-info" role="alert"><b>Note</b> that sometimes you don't get exactly the same answer as I get, but it doesn't neccessarily mean it is wrong. Could be some parameters, randomization, that we have different. Also very important is that in the future there won't be any answer sheets, use your skills in data analysis, mathematics and statistics to back up your work.</p>

<p class = "alert alert-info" role="alert"><b>Note</b> that in cases when you start to repeat code, try not to. Create functions to reuse code instead. </p>

<p class = "alert alert-info" role="alert"><b>Remember</b> to use <b>descriptive variable, function, index </b> and <b> column names</b> in order to get readable code </p>

The number of stars (\*), (\*\*), (\*\*\*) denotes the difficulty level of the task

---

## 0. Iris flower dataset (*)

In the whole exercise, we will work with the famous Iris flower dataset, which was collected in 1936 by Ronald Fisher, a statistician and biologist. Use the ```datasets``` module from scikit-learn to load the iris dataset. 

&nbsp; a) Check keys on the loaded data and check what the different values for each key are.

&nbsp; b) Now insert the data into a DataFrame. 

&nbsp; c) Do some EDA to get an understanding of the dataset. 

&nbsp; d) Make a correlation heatmap to see how each feature is correlated to each other. What do the numbers mean?

&nbsp; e) Make a boxplot. The points outside of the boxplot are statistically calculated outliers using Tukey's rule for boxplot. 

&nbsp; f) Now remove the outliers in data. (**)

- Lower bound outlier: $Q_1 - 1.5\cdot IQR$
- Upper bound outlier: $Q_3 + 1.5\cdot IQR$

where $Q_1$ is the 1st quartile or 25 percentile, $Q_3$ is the 3rd quartile or 75 percentile and $IQR = Q_3-Q_1$ is the interquartile range. 

<details>

<summary>Hint</summary>

a) For DESCR key you need to print it.

f) Dataframes has a quantile method.  

</details>

<details>

<summary>Answer</summary>

b) 

|    |   sepal length (cm) |   sepal width (cm) |   petal length (cm) |   petal width (cm) |   species | specie_name   |
|---:|--------------------:|-------------------:|--------------------:|-------------------:|----------:|:--------------|
|  0 |                 5.1 |                3.5 |                 1.4 |                0.2 |         0 | setosa        |
|  1 |                 4.9 |                3   |                 1.4 |                0.2 |         0 | setosa        |
|  2 |                 4.7 |                3.2 |                 1.3 |                0.2 |         0 | setosa        |
|  3 |                 4.6 |                3.1 |                 1.5 |                0.2 |         0 | setosa        |
|  4 |                 5   |                3.6 |                 1.4 |                0.2 |         0 | setosa        |

c) When you do describe, remove species as its statistical values are meaningless. 

|                   |    mean |      std |   min |   25% |   50% |   75% |   max |
|:------------------|--------:|---------:|------:|------:|------:|------:|------:|
| sepal length (cm) | 5.84333 | 0.828066 |   4.3 |   5.1 |  5.8  |   6.4 |   7.9 |
| sepal width (cm)  | 3.05733 | 0.435866 |   2   |   2.8 |  3    |   3.3 |   4.4 |
| petal length (cm) | 3.758   | 1.7653   |   1   |   1.6 |  4.35 |   5.1 |   6.9 |
| petal width (cm)  | 1.19933 | 0.762238 |   0.1 |   0.3 |  1.3  |   1.8 |   2.5 |


<img src = "../assets/pairplot_iris.png" height=300>

Do more EDA than I show here. 

d) Correlation heatmap

<img src = "../assets/Correlation_iris.png" height=300>

The closer the value is to 1 between two features, the more positively linear relationships between them. The closer the value is to -1 the more negatively linear relationships between them. 

e) 

<img src = "../assets/boxplot_iris.png" height=200>

f)
Outlier rows are: [13, 15, 22, 23, 24, 41, 43, 44, 98, 106, 117, 119, 131]

value counts:

|            |   specie_name |
|:-----------|--------------:|
| versicolor |            49 |
| virginica  |            46 |
| setosa     |            42 |

</details>

---

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, classification_report, mean_absolute_error, mean_squared_error, root_mean_squared_error


In [ ]:
iris = load_iris()
feature_names = iris.feature_names
target_names = iris.target_names
data = pd.DataFrame(iris.data, columns=feature_names)
df = data.copy()
df['species'] = pd.Categorical.from_codes(iris.target, target_names)
# print(iris['DESCR'])

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df_describe_T = df.describe().transpose().reset_index()
df_describe_T.drop(columns=["count"], inplace=True)
df_describe_T_melt = df_describe_T.melt(id_vars="index")

sns.barplot(data=df_describe_T_melt, x="value", y="variable", hue="index")
plt.title("iris_dataset.describe() visualised\n", fontweight="bold")
plt.ylabel("")
plt.xlabel("Amount")
plt.legend()
plt.grid(axis="x")
plt.gca().set_facecolor('whitesmoke')
plt.show()

In [ ]:
cols_numeric = df.describe().columns.to_list()
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 5))

for ax, col in zip(axes, cols_numeric):
    sns.boxplot(data=df, y=col, ax=ax, hue="species")
    ax.set_title(f'Boxplot of {col}')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(3, 3))
sns.pairplot(df, hue = "species")

In [ ]:
corr = data.corr()
sns.heatmap(corr, annot=True)

In [ ]:
def find_quantile(df, flower):

    df_flower = df[df["species"] == flower]
    df_flower = df_flower.drop(columns="species")

    q1 = df_flower.quantile(0.25)
    q3 = df_flower.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    col_list = df_flower.columns.to_list()

    for col in col_list:
        df_flower = df_flower[(df_flower[col] >= lower[col]) & (df_flower[col] <= upper[col])]
    
    df_flower["species"] = flower

    return df_flower


species_list = df["species"].unique().tolist()

df_cleaned = pd.DataFrame()
for flower in species_list:
    df_mask_cleaned = find_quantile(df, flower)
    df_cleaned = pd.concat([df_cleaned, df_mask_cleaned])

In [ ]:
def find_quantile_test(df, flower):
    df_flower = df[df["species"] == flower].drop(columns="species")
    q1 = df_flower.quantile(0.25)
    q3 = df_flower.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    for col in df_flower.columns:
        df_flower = df_flower[(df_flower[col] >= lower[col]) & (df_flower[col] <= upper[col])]
    df_flower["species"] = flower
    return df_flower

species_list = df["species"].unique().tolist()
df_cleaned_test = pd.concat([find_quantile_test(df, flower) for flower in species_list])

## 1. Split and scale data (*)

Do train|test split and scale the data using feature standardization, I used default test size 0.33 and random state 42. Check the mean and standard deviation on training and test data. 

---

In [ ]:
X = df_cleaned.drop(columns = "species")
y = df_cleaned["species"]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

df_scaled_values = pd.DataFrame({
    "mean": scaler.mean_,
    "std": np.sqrt(scaler.var_)
}, index=feature_names).T

df_scaled_values

## 2. Classify with logistic regression (*)

Use k-folded cross-validation with logistic regression to find suitable hyperparameters and model. Check the documentation to see which parameters that can be chosen through cross-validation. Check the models parameters and see what it has chosen. 

<details>

<summary>Answer</summary>

weights: 

```py
array([[-1.33033256,  1.35076961, -2.26169407, -2.07715072],
       [ 0.40073538, -0.28598722, -0.58388865, -0.7782766 ],
       [ 0.67977172, -0.81485664,  3.09503329,  3.10542664]])
```

$\ell_1$-ratio:

```py
array([0.2, 0.2, 0.2])
```


<img src = "../assets/pairplot_iris.png" height=300>

Do more EDA than I show here. 

d) Correlation heatmap

<img src = "../assets/Correlation_iris.png" height=300>

The closer the value is to 1 between two features, the more positively linear relationships between them. The closer the value is to -1 the more negatively linear relationships between them. 

e) 

<img src = "../assets/boxplot_iris.png" height=200>

f)
Outlier rows are: [13, 15, 22, 23, 24, 41, 43, 44, 98, 106, 117, 119, 131]

value counts:

|            |   specie_name |
|:-----------|--------------:|
| versicolor |            49 |
| virginica  |            46 |
| setosa     |            42 |

</details>

---

In [ ]:
model = LogisticRegressionCV(cv=5)
model.fit(X_train_scaled, y_train)
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

In [ ]:
print(f"Coefficients: \n{model.coef_}\n")

## 3. Evaluate model (*)

Make a prediction on the testing data. 

&nbsp; a) Check manually the first 10 values of $y_{test}$ against your prediction. 

&nbsp; b) Plot a confusion matrix. Can you see which predictions the model have mispredicted?

&nbsp; c) Print a classification report 

<details>

<summary>Answer</summary>


b) 

<img src = "../assets/confusion_matrix_iris.png" height=300>



c) 

Classification report 

```py
          precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      0.94      0.97        16
           2       0.94      1.00      0.97        16

    accuracy                           0.98        46
   macro avg       0.98      0.98      0.98        46
weighted avg       0.98      0.98      0.98        46
```






</details>

---

In [ ]:
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
cm_train = confusion_matrix(y_train, y_pred_train, labels = model.classes_)
cm_test = confusion_matrix(y_test, y_pred_test, labels = model.classes_)
disp_train = ConfusionMatrixDisplay(cm_train, display_labels = model.classes_)
disp_test = ConfusionMatrixDisplay(cm_test, display_labels = model.classes_)

In [ ]:
# make subplot 1x2

disp_train.plot()

In [ ]:
disp_test.plot()

In [ ]:
print(f"Training accuracy: {acc_train}\nTesting accuracy: {acc_test}\n")
print(classification_report(y_test, y_pred_test))

## 4. $k$-folded cross-validation for evaluation (**)

To be more robust in reporting the results, you should report the results as $\mu_{score}$, i.e. average score through a k-folded cross-validation. Report the score for precision, recall, f1-score for each label and overall accuracy. Do the cross-validation manually using for statement. 

In [ ]:
def calculate_scores(y_true, y_pred, split, cv):

    pre = precision_score(y_true, y_pred, average="micro")
    rec = recall_score(y_true, y_pred, average="micro")
    f1 = f1_score(y_true, y_pred, average="micro")
    acc = accuracy_score(y_true, y_pred)
    df = pd.DataFrame(
        data={
        "Precision": pre, 
        "Recall": rec, 
        "F1-score": f1, 
        "Accuracy": acc, 
        "CV": cv},
        index=pd.Index([split], name="Split")).reset_index()

    return df


def fit_predict_score(X_train, X_test, y_train, cv=5):
    
    model = LogisticRegressionCV(cv=cv)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    df_train = calculate_scores(y_train, y_pred_train, "train", cv)
    df_test = calculate_scores(y_test, y_pred_test, "test", cv)

    return pd.concat([df_train, df_test])


scores = pd.DataFrame()

for k in range(2, 11):
    scores_cv = fit_predict_score(X_train_scaled, X_test_scaled, y_train, cv=k)
    scores = pd.concat([scores, scores_cv])

scores

In [ ]:
sns.lineplot(data=scores, x="CV", y=???, hue="Split")
plt.show()

---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---